<a href="https://colab.research.google.com/github/talhaanwarch/OffenseEval2020/blob/master/Danish/biGRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [2]:
cd /content/drive/My Drive/dataset/OffenseEval2020/data/Danish/

/content/drive/My Drive/dataset/OffenseEval2020/data/Danish


In [0]:
import pandas as pd
train=pd.read_csv( 'offenseval-da-training-v1.tsv',sep="\t")

In [4]:
train.head()

,id,tweet,subtask_a
0,3131,"Jeg tror det vil være dejlig køligt, men jeg v...",NOT
1,711,Så kommer de nok til at investere i en ny cyke...,NOT
2,2500,Nu er det jo også de Ikea-aber der har lavet s...,OFF
3,2678,"128 Varme emails, er vi enige om at det er sex...",NOT
4,784,"Desværre tyder det på, at amerikanerne er helt...",NOT


In [5]:
train.tail()

,id,tweet,subtask_a
2956,1226,"Ind og ruske tremmer med hende,Den syge kælling!!",OFF
2957,2596,fedtmule,NOT
2958,1802,##HAR I HØRT DET?,NOT
2959,2809,Kommer det bag på nogen? Det er jo fucking var...,OFF
2960,4000 [deleted] NOT,NaN,NaN


In [0]:
train.drop(train.tail(n=1).index,inplace=True)

In [7]:
train.tail()

,id,tweet,subtask_a
2955,170,Har sgu lidt en anelse om... det her kunne mås...,NOT
2956,1226,"Ind og ruske tremmer med hende,Den syge kælling!!",OFF
2957,2596,fedtmule,NOT
2958,1802,##HAR I HØRT DET?,NOT
2959,2809,Kommer det bag på nogen? Det er jo fucking var...,OFF


In [0]:
labels=train['subtask_a']
train=train['tweet']
labels=pd.factorize(labels)[0]

In [9]:
import collections
collections.Counter(labels)

Counter({0: 2576, 1: 384})

In [10]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
max_words = 10000 #frequency of words to be kept
max_len = 200

tokenize = Tokenizer(num_words=max_words)
tokenize.fit_on_texts(train)
sequences = tokenize.texts_to_sequences(train)
word_index = tokenize.word_index
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

Using TensorFlow backend.


In [11]:
len(sequences_matrix),len(labels)

(2960, 2960)

In [0]:
from keras.models import Sequential
from keras.layers import Embedding,CuDNNGRU,GRU,Dense,Dropout,Bidirectional,GlobalMaxPool1D,GlobalAveragePooling1D, SpatialDropout1D
from keras.optimizers import RMSprop
from keras.initializers import Constant
from sklearn.utils import class_weight
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

In [0]:
from keras import backend as K
def f1(y_true, y_pred):
    '''
    metric from here 
    https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras
    '''
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [0]:
model = Sequential()
model.add(Embedding(input_dim=max_words,output_dim=50,input_length=max_len))
model.add(Bidirectional(CuDNNGRU(64, return_sequences = True)))
#model.add(Bidirectional(CuDNNGRU(30, return_sequences = True)))
model.add(GlobalMaxPool1D())
model.add(Dense(50, activation="relu"))
model.add(Dense(20, activation="relu"))

model.add(Dropout(0.05))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[f1])

In [0]:
import numpy as np
class_weights = class_weight.compute_class_weight('balanced',np.unique(labels),labels)
class_weights=dict(enumerate(class_weights))

In [16]:

model.fit(sequences_matrix,labels,batch_size=256,epochs=10,verbose=2,
          validation_split=0.2,class_weight=class_weights)

Train on 2368 samples, validate on 592 samples
Epoch 1/10
 - 3s - loss: 0.6907 - f1: 0.2315 - val_loss: 0.7004 - val_f1: 0.2451
Epoch 2/10
 - 1s - loss: 0.6875 - f1: 0.2442 - val_loss: 0.6986 - val_f1: 0.2525
Epoch 3/10
 - 1s - loss: 0.6797 - f1: 0.2995 - val_loss: 0.6937 - val_f1: 0.3018
Epoch 4/10
 - 1s - loss: 0.6594 - f1: 0.5100 - val_loss: 0.6803 - val_f1: 0.3767
Epoch 5/10
 - 1s - loss: 0.5981 - f1: 0.7168 - val_loss: 0.6392 - val_f1: 0.4062
Epoch 6/10
 - 1s - loss: 0.4520 - f1: 0.7934 - val_loss: 0.6105 - val_f1: 0.5031
Epoch 7/10
 - 1s - loss: 0.2843 - f1: 0.8577 - val_loss: 0.6540 - val_f1: 0.5045
Epoch 8/10
 - 1s - loss: 0.1570 - f1: 0.9348 - val_loss: 0.5851 - val_f1: 0.5166
Epoch 9/10
 - 1s - loss: 0.0623 - f1: 0.9776 - val_loss: 0.7696 - val_f1: 0.5455
Epoch 10/10
 - 1s - loss: 0.0241 - f1: 0.9812 - val_loss: 1.0143 - val_f1: 0.5414


In [0]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val=train_test_split(sequences_matrix,labels,test_size=0.2,stratify=labels)

In [18]:
from sklearn.metrics import classification_report

y_pred = model.predict(x_val, batch_size=128, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
y_pred = (y_pred > 0.5)

print(classification_report(y_val, y_pred))

592/592 [==============================] - 0s 331us/step
              precision    recall  f1-score   support

           0       0.98      0.99      0.98       515
           1       0.91      0.88      0.89        77

    accuracy                           0.97       592
   macro avg       0.94      0.93      0.94       592
weighted avg       0.97      0.97      0.97       592



In [0]:
import keras
keras.backend.clear_session()